# Librarii importate

In [20]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split


# Date din fisier declarate manual intr-un dictionar

In [21]:
data = {
    'R': [255, 0, 0, 255, 0, 0, 192, 128, 0, 25,122],
    'G': [0, 255, 0, 255, 255, 0, 192, 128, 0, 255,122],
    'B': [0, 0, 255, 0, 255, 255, 192, 128, 0, 255,122],
    'Color Name': ['Red', 'Green', 'Blue', 'Yellow', 'Cyan', 'Magenta', 'Silver', 'Gray', 'Black', 'White','HHHH']
}


# Functii pentru manipularea fisierului csv

In [22]:
def create_csv(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

    df = pd.DataFrame(data)
    df.to_csv(file_path, index=False)
    print(f"'{file_path}' has been created.")

def load_data(file_path):
    data = pd.read_csv(file_path)
    X = data.iloc[:, :3].values
    y = data.iloc[:, 3].values
    return X, y

# Preprocesare


In [23]:
def preprocess_data(X, y):
    from sklearn.preprocessing import LabelEncoder
    from tensorflow.keras.utils import to_categorical

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # bam bam bam integeri
    y_categorical = to_categorical(y_encoded)  # One-hot
    return X / 255.0, y_categorical, label_encoder

# Model

In [24]:
def build_model(input_shape, num_classes):
    import tensorflow as tf

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='sigmoid', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='sigmoid'),
        tf.keras.layers.Dense(num_classes, activation='relu')
    ])
    model.compile(optimizer='RMSprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Functie de antrenare

In [25]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size
    )
    return history

# Functia main()

In [26]:
def main(file_path):
    create_csv(file_path)
    X, y = load_data(file_path)
    X_preprocessed, y_categorical, label_encoder = preprocess_data(X, y)
    X_train, X_val, y_train, y_val = train_test_split(X_preprocessed, y_categorical, test_size=0.2, random_state=69)

    input_shape = (X_train.shape[1],)
    num_classes = y_categorical.shape[1]
    model = build_model(input_shape, num_classes)

    history = train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32)

    # SALVAM MODELUL
    model.save('trained_model.keras')

    print("Training complete. Model saved to 'trained_model.keras'.")


In [ ]:
main('colors.csv')